### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from neuropy import plotting
from scipy.ndimage import gaussian_filter1d
from neuropy import plotting
from scipy import stats
import subjects
import pandas as pd

### PRE vs POST firing rate scatter plot 

In [ ]:
sessions = subjects.mua_sess()

In [ ]:
mean_frate_all = []
for sub, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons_stable.get_neuron_type("pyr")
    artifact = sess.artifact

    mean_frate=[]
    for e in [pre, post]:
        artifact_duration = (
            0 if artifact is None else artifact.time_slice(*e).durations.sum()
        )
        n_spikes = neurons.time_slice(*e).n_spikes
        mean_frate.append(n_spikes / (np.diff(e) - artifact_duration))
    mean_frate_all.append(np.array(mean_frate))

mean_frate_all = np.hstack(mean_frate_all)

In [ ]:
_,ax = plt.subplots()

ax.scatter(mean_frate_all[0],mean_frate_all[1],s=8)
ax.axline((0,0),slope=1)
ax.axline((1,1.5),slope=1)
ax.axline((1,0.5),slope=1)
ax.set_xscale('log')
ax.set_yscale('log')

### Firing rate stability visualization
- something like Miyawaki et al. 2016

In [2]:
sessions = subjects.nsd.ratRday1

In [4]:
frate_all=[]
for sub, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type(["pyr",'inter','mua'])
    
    rec_duration = sess.eegfile.duration
    windows = np.linspace(0,rec_duration,6)
    n_windows = len(windows)-1
    epochs = np.vstack([windows[:-1],windows[1:]]).T

    mean_frate = neurons.firing_rate[:,np.newaxis]

    frate = []
    for i,e in enumerate(epochs):
        if sess.artifact is not None: 
            artifact_duration = sess.artifact.time_slice(*e,strict=False).durations.sum()
        else:
            artifact_duration = 0
        n_spikes = neurons.time_slice(*e).n_spikes
        e_duration = e[1]-e[0]
        frate.append(n_spikes / (e_duration - artifact_duration))

    frate = np.array(frate).T 
    frate_perc = frate/mean_frate

    stable_bool = np.count_nonzero((frate_perc>(1/4)),axis=1)==n_windows
    stable_neurons = neurons[stable_bool]
    stable_neurons.save(sess.filePrefix.with_suffix('.neurons_stable2'))
    frate_all.append(frate_perc)

frate_all = np.vstack(frate_all)

/data/Clustering/sessions/RatR/Day1NSD/RatR_Day1NSD_2021-05-13_08-41-31.neurons_stable2 saved


In [ ]:
_,ax = plt.subplots()

ax.plot(frate_all.T,'o-')
# ax.set_yscale('log')

### Scatter plot for firing rate
- Something similar to supplementary figure of Miyawaki2016

In [ ]:
sessions = subjects.mua_sess()

In [ ]:
frate_df = []
for sub, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    maze = sess.paradigm["maze"].flatten()
    post = sess.paradigm["post"].flatten()
    neurons = sess.neurons.get_neuron_type('pyr')
    neurons_id = neurons.neuron_ids
    stable_neurons_id = sess.neurons_stable.get_neuron_type("pyr").neuron_ids

    rec_duration = sess.eegfile.duration
    windows = np.linspace(0, rec_duration, 6)
    n_windows = len(windows) - 1
    epochs = np.vstack([windows[:-1], windows[1:]]).T

    mean_frate = neurons.firing_rate[:, np.newaxis]

    frate = []
    for i, e in enumerate(epochs):
        if sess.artifact is not None:
            artifact_duration = sess.artifact.time_slice(
                *e, strict=False
            ).durations.sum()
        else:
            artifact_duration = 0
        n_spikes = neurons.time_slice(*e).n_spikes
        e_duration = e[1] - e[0]
        frate.append(n_spikes / (e_duration - artifact_duration))

    frate = np.array(frate).T
    df = pd.DataFrame(frate, columns=np.arange(5))
    df['mean_frate'] = mean_frate
    df['stable'] = np.isin(neurons_id,stable_neurons_id,assume_unique=True)
    df["grp"] = sess.tag
    frate_df.append(df)

frate_df = pd.concat(frate_df, ignore_index=True)


In [ ]:
fig = plotting.Fig(grid=(8,5))

kw = dict(hue='stable',palette=['#22d354','#c6d2c9'],hue_order=[True,False])

ax =fig.subplot(fig.gs[0])
df = frate_df.sample(n=5,replace=False)
df.iloc[:,:5] = df.iloc[:,:5].div(df.mean_frate,axis=0)*100
frate_df_long = df.reset_index()
frate_df_long = pd.melt(
    frate_df_long,
    id_vars=["grp",'index','stable'],
    value_vars=np.arange(5),
    var_name=["epochs"],
    value_name="frate",
    ignore_index=True,
)

sns.lineplot(data=frate_df_long,x='epochs',y='frate',units='index',estimator=None,marker='o',ax=ax,**kw)
ax.axhline(25,color='gray',lw=2)
# ax.set_yscale('log')
ax.set_ylim(0,400)
ax.legend('',frameon=False)

for i,(e1,e2) in enumerate(zip([0,2,4],[2,4,0])):
    ax = fig.subplot(fig.gs[i+1])
    sns.scatterplot(data=frate_df,x=e1,y=e2,s=7,**kw,ax=ax,legend=False)
    ax.set_xscale('log')
    ax.set_yscale('log')

fig.savefig(subjects.figpath_sd/'frate_stability')


### Firing rate stability using gini coefficient

In [ ]:
sessions = subjects.mua_sess()

In [ ]:
from neuropy.utils.mathutil import gini
frate_all = []
for sub, sess in enumerate(sessions):
    pre = sess.paradigm["pre"].flatten()
    post = sess.paradigm["post"].flatten()

    rec_duration = sess.eegfile.duration
    windows = np.linspace(0, rec_duration, 10)
    neurons = sess.neurons.get_neuron_type(["pyr", "inter"])
    artifact = sess.artifact

    frate_pre = neurons.time_slice(*pre).firing_rate
    frate_post = neurons.time_slice(*post).firing_rate

    frate = []
    for i in np.arange(len(windows) - 1):
        e = windows[i : i + 2]
        artifact_duration = (
            0 if artifact is None else artifact.time_slice(*e).durations.sum()
        )
        n_spikes = neurons.time_slice(*e).n_spikes
        frate.append(n_spikes / (np.diff(e) - artifact_duration))
    frate = np.array(frate).T
    g_coeffs = np.array([gini(_) for _ in frate])

    stable_bool = (g_coeffs <= 0.7) & (frate_pre > 0) & (frate_post > 0)

    stable_neurons = neurons[stable_bool]
    stable_neurons.save(sess.filePrefix.with_suffix(".neurons_stable"))
